In [67]:
####Question 3c Prepare rating matrix
import pandas as pd
import math

users = pd.read_pickle('q3c/q3c_female_profiles.pkl',compression='gzip')
history = pd.read_pickle('q3c/q3c_female_history.pkl',compression='gzip')

all_artist_name = history['artist-name'].unique().tolist()
all_users = users['user-mboxsha1'].unique().tolist()



frequency = history.groupby('artist-name')['plays'].sum().sort_values()

####artists with play frequencies greater than 100000 will be considered popular artists
popular_artist = frequency[frequency > 100000]
popular_artist_df = pd.DataFrame({'freq':popular_artist})
filtered_hist = history.set_index('artist-name').loc[popular_artist_df.index.tolist()].reset_index()


filtered_hist = filtered_hist.groupby(['user-mboxsha1','artist-name'])['plays'].sum().reset_index()

rating_matrix = filtered_hist.pivot(index='artist-name',\
                                      columns='user-mboxsha1',\
                                      values='plays')

def compute_average(row):
    notnull = row[row.notnull()]
    average_rating = row.sum()/len(notnull)    
    return row - average_rating
rating_matrix['_AVERAGE'] = rating_matrix.apply(compute_average,axis=1)

rating_matrix.to_pickle('q3c/q3c_rating_matrix.pkl',compression='gzip')

print('finish producing rating matrix')



finish producing rating matrix


In [4]:
#####Prepare normalized rating matrix
import pandas as pd
rating_matrix = pd.read_pickle('q3c/q3c_rating_matrix.pkl',compression='gzip')

def normalize_values(row):
    result = row - row['_AVERAGE']
    del result['_AVERAGE']
    return result

print('start producing normalized rating matrix')
normalized_rating_matrix = rating_matrix.apply(normalize_values,axis=1)
print('finish producing normalized rating matrix')

normalized_rating_matrix.to_pickle('q3c/q3c_normalized_rating_matrix.pkl',compression='gzip')



start producing normalized rating matrix
finish producing normalized rating matrix


In [12]:
#####prepare similarity matrix using cosine similarity and normalized rating matrix
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

normalized_rating_matrix = pd.read_pickle('q3c/q3c_normalized_rating_matrix.pkl',compression='gzip')

all_artist_name = normalized_rating_matrix.index.unique().tolist()

similarity_matrix = np.zeros((len(all_artist_name),len(all_artist_name)) )
similarity_matrix = pd.DataFrame(similarity_matrix,columns=all_artist_name,index=all_artist_name)

for i,an_i in enumerate(all_artist_name):
    all_artist_name_j = all_artist_name[i+1:]
    vec_i = normalized_rating_matrix.loc[an_i].fillna(0)
    vec_i = np.array(vec_i).reshape((1,len(vec_i )))
    similarity_matrix[an_i][an_i] = 1
    for j,an_j in enumerate(all_artist_name_j):
        vec_j = normalized_rating_matrix.loc[an_j].fillna(0)
        vec_j = np.array(vec_j).reshape((1,len(vec_j )))
        vec_similarity = cosine_similarity(vec_i,vec_j)
        similarity_matrix[an_j][an_i] = vec_similarity
        similarity_matrix[an_i][an_j] = vec_similarity
    print('finish {}'.format(i))



similarity_matrix.to_pickle('q3c/q3c_similarity_matrix.pkl',compression='gzip')

    




finish 0
finish 1
finish 2
finish 3
finish 4
finish 5
finish 6
finish 7
finish 8
finish 9
finish 10
finish 11
finish 12
finish 13
finish 14
finish 15
finish 16
finish 17
finish 18
finish 19
finish 20
finish 21
finish 22
finish 23
finish 24
finish 25
finish 26
finish 27
finish 28
finish 29
finish 30
finish 31
finish 32
finish 33
finish 34
finish 35
finish 36
finish 37
finish 38
finish 39
finish 40
finish 41
finish 42
finish 43
finish 44
finish 45
finish 46
finish 47
finish 48
finish 49
finish 50
finish 51
finish 52
finish 53
finish 54
finish 55
finish 56
finish 57
finish 58
finish 59
finish 60
finish 61
finish 62
finish 63
finish 64
finish 65
finish 66
finish 67
finish 68
finish 69
finish 70
finish 71
finish 72
finish 73
finish 74
finish 75
finish 76
finish 77
finish 78
finish 79
finish 80
finish 81
finish 82
finish 83
finish 84
finish 85
finish 86
finish 87
finish 88
finish 89
finish 90
finish 91
finish 92
finish 93
finish 94
finish 95
finish 96
finish 97
finish 98
finish 99
finish 100

finish 755
finish 756
finish 757
finish 758
finish 759
finish 760
finish 761
finish 762
finish 763
finish 764
finish 765
finish 766
finish 767
finish 768
finish 769
finish 770
finish 771
finish 772
finish 773
finish 774
finish 775
finish 776
finish 777
finish 778
finish 779
finish 780
finish 781
finish 782
finish 783
finish 784
finish 785
finish 786
finish 787
finish 788
finish 789
finish 790
finish 791
finish 792
finish 793
finish 794
finish 795
finish 796
finish 797
finish 798
finish 799
finish 800
finish 801
finish 802
finish 803
finish 804
finish 805
finish 806
finish 807
finish 808
finish 809
finish 810
finish 811
finish 812
finish 813
finish 814
finish 815
finish 816
finish 817
finish 818
finish 819
finish 820
finish 821
finish 822
finish 823
finish 824
finish 825
finish 826
finish 827
finish 828
finish 829
finish 830
finish 831
finish 832
finish 833
finish 834
finish 835
finish 836
finish 837
finish 838
finish 839
finish 840
finish 841
finish 842
finish 843
finish 844
finish 845

In [44]:
######building the actual recommendation engine using similarity matrix and normalized rating matrix
import pandas as pd

normalized_rating_matrix = pd.read_pickle('q3c/q3c_normalized_rating_matrix.pkl',compression='gzip')
similarity_matrix = pd.read_pickle('q3c/q3c_similarity_matrix.pkl',compression='gzip')

all_artist_name = normalized_rating_matrix.index.unique().tolist()
all_female_users = normalized_rating_matrix.columns.tolist()


def recommendation_engine(uid,max_number_of_recommendations=5,k_nearest_neighbors=5):
    recommendations = []
    predicted_ratings_df = pd.Series()
    if uid in normalized_rating_matrix.columns:
        users_rating = normalized_rating_matrix[uid]
        unlistened_artists = users_rating[users_rating.isnull()]
        listened_artists = users_rating[users_rating.notnull()]
        
        #####step 1: compute ratings for all unlistened artists
        for artist in unlistened_artists.index.tolist():
            ####get all cosine similarities for current unlistened artist
            artist_similarity_vector = similarity_matrix.loc[artist]
            
            ####only keep cosine similarities for listened artists
            artist_similarity_vector = artist_similarity_vector[listened_artists.index.tolist()].sort_values()
            
            ####filter out the current unlistened artist
            artist_similarity_vector =\
                artist_similarity_vector[artist_similarity_vector.index != artist] 
            
            ####only keep top k_nearest_neighbors similar artists
            similar_artists = artist_similarity_vector.tail(k_nearest_neighbors)
            
            #####predict normalized rating for current unlistened artist using weighted average
            total_similarity = similar_artists.sum()
            similar_artists_ratings = listened_artists[similar_artists.index.tolist()]
            predicted_rating = (similar_artists_ratings * similar_artists).sum()/total_similarity
            predicted_ratings_df[artist] = predicted_rating
        
        #####step 2: sort unlistened artists ratings and 
        #####get the top max_number_of_recommendations ratings as recommendations
        predicted_ratings_df = predicted_ratings_df.sort_values()
        recommendations = predicted_ratings_df.tail(max_number_of_recommendations).index.tolist()
        
    return recommendations
            
        
#####Example
test_user = all_female_users[0]

print(recommendation_engine(test_user))

['madsen', 'die toten hosen', 'seeed', 'beatsteaks', 'farin urlaub']


In [ ]:
"""
Design of the recommendation system
My implementation of the recommendation system will return a list of artists for a female user id.
The first step is to extract to female users from usersha1-profile.tsv(I didnt include the original tsv files because the files are huge) and I stored them into q3c_female_profiles.pkl. 
Then, I extract all listening history for these female users from usersha1-artmbid-artname-plays.tsv. 
After that, I need to prepare the rating matrix where the rows are artists and columns are the female user ids.
Since there are too many artists, I only consider artists with acculative play frequencies greater than 100000.
After creating the rating matrix, I need to build the normalized rating matrix by substracting the average rating from the original rating for each artist.
After that, I build a similarity matrix where the rows and columns are the popular artists.
Each entry in the similarity matrix is calculated using normalized rating matrix and the cosine similarity.
The similarity matrix is symmetric. Entry[i][j] is the cosine similarity between artist i and artist j.
Everything required to build my recommendation engine is ready. 
The engine will take a female user id. Then, it will find the list of artists that the female user already listened to and the list of artists that the female never listened to.
At this point, the goal is to fill in the ratings for the unlistened artists. 
By using the similarity matrix, I can find the similar artists and I compute the rating by using the weighted average.
After computing all missing ratings, I sort them and extract the top ratings as recommendations.


Limitations:
Currently, the recommendation engine will only recommend popular artists (subset of artists). 
In production, it should consider all artists.
"""